## Import data

In [1]:
import os, pandas as pd, numpy as np
from DpdDataParsing import *

In [8]:
folder ="C:\\Users\\Micky\\Desktop\\data\\"
X = pd.DataFrame()
start_time = time.time()
for csv in os.listdir(folder): 
    print(f"Reading . . . <{csv}>")
    other = create_mob(pd.read_csv(f"{folder}{csv}"), 
                       dt_fmt="%Y-%m-%d %H:%M:%S", digit=2) 
    X = X.append(other, ignore_index=True)
    del other
r_time = time.gmtime(time.time() - start_time)
r_time = time.strftime("%H:%M:%S", r_time)
print('Total running time: {}'.format(r_time))
print("Complete")

Reading . . . <delinquent_application_202101_202206.csv>
Total running time: 00:00:03
Reading . . . <delinquent_application_202102_202206.csv>
Total running time: 00:00:08
Reading . . . <delinquent_application_202103_202206.csv>
Total running time: 00:00:15
Reading . . . <delinquent_application_202104_202206.csv>
Total running time: 00:00:13
Reading . . . <delinquent_application_202105_202206.csv>
Total running time: 00:00:09
Reading . . . <delinquent_application_202106_202206.csv>
Total running time: 00:00:11
Reading . . . <delinquent_application_202107_202206.csv>
Total running time: 00:00:09
Reading . . . <delinquent_application_202108_202206.csv>
Total running time: 00:00:07
Reading . . . <delinquent_application_202109_202206.csv>
Total running time: 00:00:07
Reading . . . <delinquent_application_202110_202206.csv>
Total running time: 00:00:05
Reading . . . <delinquent_application_202111_202206.csv>
Total running time: 00:00:07
Reading . . . <delinquent_application_202112_202206.cs

In [9]:
X.shape

(1569413, 45)

In [10]:
dpd_cols = X.columns[X.columns.str.contains(pat="M.[0-9]$")].tolist()
X["clean"] = X[dpd_cols].fillna(0).sum(1)==0

In [11]:
X["cohort"] = X["fnl_apl_dcsn_dt"].astype(str).apply(lambda x:x[:7])
X["util"] = np.clip(np.fmax(X["M00_PNP"], X["M01_PNP"]) / X["fnl_cr_lmt"], 0, 1)

Change channel categories.

In [12]:
cat = {"DS":"Self-Apply", "EA":"E_App", "QR":"Paper"}
X["apl_grp_type"] = X["apl_grp_type"].apply(lambda x:cat[x[:2]])

Binned FICO scores.

In [13]:
X["fico_scor"] = X["fico_scor"].fillna(0)
bins = np.r_[[0,1],np.arange(300,1000,50)]
fico_categories = dict([(n+1,"{}<=x<{}".format(*bins[n:n+2])) 
                        for n in np.arange(len(bins)-1)])
X["fico_bins"] = np.digitize(X["fico_scor"].fillna(0), bins)
X["fico_bins"] = X["fico_bins"].apply(lambda x : fico_categories[x])

Binned utilizations.

In [14]:
bins =np.round(np.arange(0, 1.1, 0.1),2)
util_categories = dict([(n+1,"{}<=x<{}".format(*bins[n:n+2])) 
                        for n in np.arange(len(bins)-1)])
util_categories[len(bins)] = "1<=x<inf"
util_categories[0] = "-inf<=x<0"
X["util_bins"] = np.digitize(X["util"].fillna(0), bins)
X["util_bins"] = X["util_bins"].apply(lambda x : util_categories[x])

In [15]:
X.head()

,ip_id,pd_lvl2,cust_type,apl_grp_no,apl_grp_type,fnl_apl_dcsn_dt,fnl_cr_lmt,fico_scor,M00,M01,...,M15_OS,M16_OS,M00_PNP,M01_PNP,M02_PNP,clean,cohort,util,fico_bins,util_bins
0,68,KPL,Salary Earner,QR-7225183,Paper,2021-01-31,11000.0,0.0,0.0,0.0,...,4500.16,3988.34,11000.0,10565.75,10168.38,True,2021-01,1.0,0<=x<1,1<=x<inf
1,988,CC,Salary Earner,QR-6898548,Paper,2021-01-31,99000.0,0.0,0.0,0.0,...,3030.13,590.00,-1.0,0.00,-7200.24,True,2021-01,0.0,0<=x<1,0.0<=x<0.1
2,2327,KPL,Salary Earner,EA-210109000079,E_App,2021-01-31,32000.0,0.0,0.0,0.0,...,NaN,NaN,32000.0,26831.98,24220.08,True,2021-01,1.0,0<=x<1,1<=x<inf
3,2481,KPL,Salary Earner,QR-7228901,Paper,2021-01-31,80000.0,0.0,0.0,0.0,...,66177.34,65121.47,80000.0,79029.45,78095.09,True,2021-01,1.0,0<=x<1,1<=x<inf
4,3098,CC,Salary Earner,QR-7242777,Paper,2021-01-31,23000.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.00,True,2021-01,0.0,0<=x<1,0.0<=x<0.1


In [16]:
X.to_csv('cohort_202101_202205.txt', sep="|", index=False)